In [177]:
!pip install py-bgg
!pip install oauth2client
!pip install PyOpenSSL
!pip install gspread
!pip install df2gspread

     |████████████████████████████████| 56 kB 5.0 MB/s eta 0:00:011


  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11953 sha256=24c55e1fc7fb8663ac28b66cf2bfacb1530ac5d3d6fb54b66c9b212e35913a8e
  Stored in directory: /Users/EQ81TW/Library/Caches/pip/wheels/01/9b/3f/0aadc61c8368949be224ea67569d16aa599018edce1afe476f
Successfully built df2gspread


In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import json
from df2gspread import df2gspread as d2g, gspread2df as g2d
import pandas as pd
from libbgg.apiv2 import BGG as BGG2
import requests
from bs4 import BeautifulSoup
import re
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [11]:
class GameDatabase:
    def __init__(self):
        self.keyfile = "../boardgamegeek-370306-5094bf07e07e.json"
        self.spreadsheet_key = "1DlFqPX0AyugOsQPLYVty2rwECEmiWb4IoSPr2HOSAbI"
        self._games_cols = [
            "type",
            "id",
            "thumbnail",
            "image",
            "name",
            "description",
            "year",
            "suggested_players",
            "suggested_age",
            "official_age",
            "min_playingtime",
            "max_playingtime",
            "rating",
            "weight",
            "geekrating",
            "stddev",
            "votes",
            "rank",
            "boardgamemechanic",
        ]
        self.setup_connection()
    
    def setup_connection(self):
        scopes = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]
        self.credentials = ServiceAccountCredentials.from_json_keyfile_name(self.keyfile, scopes)
        self.gc = gspread.authorize(self.credentials)
        
    def _load_dfs_from_google_sheets(self):
        self.games_df = g2d.download(spreadsheet_key, wks_name="games", col_names=True, credentials=credentials)
        
    def load():
        self._load_dfs_from_google_sheets()
        
    def _save_dfs_to_google_sheets(self):
        self.games_df = self.games[self._games_cols]
        self.suggested_players_df = (
            self.games[["id", "suggested_players"]]
            .explode("suggested_players")
            .reset_index(drop=True)
        )
        self.suggested_players_df.columns = ["id", "suggested_player"]
        df = self.games_df.merge(self.suggested_players_df, on = "id", how = "left")
        print(len(df))
        d2g.upload(df, self.spreadsheet_key, wks_name="games", credentials=self.credentials, row_names=False)

    def save(self):
        self._save_dfs_to_google_sheets()

In [3]:
class BoardGame():
    max_percentage_not_recommended = 33
    conn = BGG2()
    
    def __init__(self, scrape_id=None):
        if scrape_id:
            self.parse(self.conn.boardgame(scrape_id, stats=True)["items"]["item"])
    
    def parse(self, data):
        self.type = data["type"]
        self.id = int(data["id"])
        self.thumbnail = data["thumbnail"]["TEXT"] if data.get("thumbnail") is not None else None
        self.image = data["image"]["TEXT"] if data.get("image") is not None else None
        self.name = self.get_name(data["name"])
        self.description = data["description"]["TEXT"]
        self.year = int(data["yearpublished"]["value"])
        self.official_players = list(range(int(data["minplayers"]["value"]), int(data["maxplayers"]["value"])+1))
        self.suggested_players = self.get_suggested_players(data["poll"])
        self.suggested_age = self.get_suggested_age(data["poll"])
        self.official_age = int(data["minage"]["value"])
        self.min_playingtime = int(data["minplaytime"]["value"])
        self.max_playingtime = int(data["maxplaytime"]["value"])
        self.process_links(data["link"])
        self.process_statistics(data["statistics"]["ratings"])
        
    @staticmethod
    def get_name(names):
        if type(names) == list:
            return names[0]["value"]
        else:
            return names["value"]
        
    def __str__(self):
        return json.dumps(vars(self), sort_keys=True, indent=4)
    
    def get_suggested_players(self, polls):
        suggested_players = []
        for poll in polls:
            if poll["name"] == "suggested_numplayers":
                # Deal with some weird games with broken numplayers
                if type(poll["results"]) != list:
                    results = [poll["results"]]
                else:
                    results = poll["results"]
                for result in results:
                    players = result["numplayers"]
                    if "+" in players:
                        players = int(players[:-1])+1
                    else:
                        players = int(players)
                    total_votes = sum([int(r["numvotes"]) for r in result["result"]])
                    not_recommended_votes = int(result["result"][2]["numvotes"])
                    if total_votes > 0 and \
                        not_recommended_votes / total_votes*100 <= self.max_percentage_not_recommended:
                        suggested_players.append(players)
                break
        return suggested_players
            
    def get_suggested_age(self, polls):
        for poll in polls:
            if poll["name"] == "suggested_playerage":
                return self.median(
                    {
                        int(d["value"].split(" ")[0]): int(d["numvotes"]) 
                        for d in poll["results"]["result"]
                    }
                )
    
    @staticmethod
    def median(histogram):
        total = 0
        median_index = (sum(histogram.values()) + 1) / 2
        for value in sorted(histogram.keys()):
            total += histogram[value]
            if total >= median_index:
                return value
            
    def process_links(self, links):
        self.boardgamecategory = []
        self.boardgamemechanic = []
        self.boardgamefamily = []
        self.boardgamedesigner = []
        self.boardgamepublisher = []
        self.boardgameartist = []
        self.boardgameexpansion = []
        self.boardgameintegration = []
        for link in links:
            if link["type"] == "boardgamecategory":
                self.boardgamecategory.append(link["value"])
            elif link["type"] == "boardgamemechanic":
                self.boardgamemechanic.append(link["value"])
            elif link["type"] == "boardgamefamily":
                self.boardgamefamily.append(link["value"])
            elif link["type"] == "boardgamedesigner":
                self.boardgamedesigner.append(link["value"])
            elif link["type"] == "boardgamepublisher":
                self.boardgamepublisher.append(link["value"])
            elif link["type"] == "boardgameartist":
                self.boardgameartist.append(link["value"])
            elif link["type"] == "boardgameexpansion":
                self.boardgameexpansion.append(link["value"])
            elif link["type"] == "boardgameintegration":
                self.boardgameintegration.append(link["value"])
                
    def process_statistics(self, stats):
        self.rating = float(stats["average"]["value"])
        self.weight = float(stats["averageweight"]["value"])
        self.geekrating = float(stats["bayesaverage"]["value"])
        self.stddev = float(stats["stddev"]["value"])
        self.votes = int(stats["usersrated"]["value"])
        if type(stats["ranks"]["rank"]) == list:
            self.rank = int(stats["ranks"]["rank"][0]["value"])
        else:
            self.rank = int(stats["ranks"]["rank"]["value"])
    
    def to_series(self):
        return pd.Series(vars(self))

In [4]:
class BGGScraper():
    def __init__(self, games=dict()):
        self.conn = BGG2()
        self.games = games
        
    @staticmethod
    def get_games(page=1):
        url = f"https://boardgamegeek.com/browse/boardgame/page/{page}"
        cookies = dict(
            bggpassword = "1746vs5j5xr8s8bw3e666yliysag310k6",
            bggusername = "tijlk",
        )
        reqs = requests.get(url, cookies=cookies)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        ids = []
        for link in soup.find_all('a'):
            if link.get('href'):
                match = re.search(r"\/boardgame\/([0-9]+)\/", link.get('href'))
                if match:
                    if int(match[1]) not in ids:
                        ids.append(int(match[1]))
        return ids

    def scrape_game_info(self, ids=None):
        if type(ids) == int:
            game = BoardGame()
            game.parse(conn.boardgame(ids, stats=True)["items"]["item"])
            return game
        elif type(ids) == list:
            data = BGG2().boardgame(ids, stats=True)["items"]["item"]
            games = []
            for game_info in data:
                game = BoardGame()
                try:
                    game.parse(game_info)
                except:
                    print(f"\n   {game_info.id}\n")
                    print(json.dumps(game_info, sort_keys=True, indent=4))
                    raise Exception
                games.append(game)
            return games
    
    def scrape_page(self, page=1):
        ids = self.get_games(page=page)
        filtered_ids = [gameid for gameid in ids if gameid not in self.games]
        if len(filtered_ids) > 0:
            game_infos = self.scrape_game_info(filtered_ids)
            for game_info in game_infos:
                self.games[game_info.id] = game_info
                
    def games_to_df(self):
        return pd.DataFrame([game.to_series() for gameid, game in scraper.games.items()])

In [5]:
scraper = BGGScraper()
for page in range(1,41):
    print(f"Page {page}...")
    scraper.scrape_page(page)
    time.sleep(2)

Page 1...
Page 2...
Page 3...
Page 4...
Page 5...
Page 6...
Page 7...
Page 8...
Page 9...
Page 10...
Page 11...
Page 12...
Page 13...
Page 14...
Page 15...
Page 16...
Page 17...
Page 18...
Page 19...
Page 20...
Page 21...
Page 22...
Page 23...
Page 24...
Page 25...
Page 26...
Page 27...
Page 28...
Page 29...
Page 30...
Page 31...
Page 32...
Page 33...
Page 34...
Page 35...
Page 36...
Page 37...
Page 38...
Page 39...
Page 40...


In [12]:
db = GameDatabase()
db.games = scraper.games_to_df()
db.save()

12088
